# Übung 3

In [8]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

**Aufgabe 11**

**A:** Für den Datensatz in Unfaelle.txt wird folgendes logistisches Regressionsmodell
$$M : \text{Unfall} ∼ \text{Alter} + \text{Fahrpraxis} + \text{Beruf} + \text{Geschlecht}\text{ (1)}$$
betrachtet. Durch Ausdrücke wie (1) werden in R Regressionsmodelle
unterschiedlichster Art spezifiziert. Geben Sie die Modellgleichung von
M in mathematischer Notation an. Wie viele Parameter werden benötigt?
Hinweis: Die Variablen Beruf und Geschlecht müssen zunächst in Dummy-
Variablen umkodiert werden.

In unserem Datensatz gibt es 3 Berufe:
- Physiker
- Zahnarzt
- Biologe

Sowie die beiden Geschlechter
- Mann
- Frau

Demzufolge brauchen wir jeweils für den ersten Fall 2, und den zweiten Fall eine Dummy-Variable.
$$
\text{Unfall} \log \Bigg ( \frac{P(Unfall=1)}{1 - P(Unfall=1)} \Bigg )= \beta_0 + \beta_1 \text{ Alter} + \beta_2 \text{ Fahrpraxis} + \beta_3D_{3i} + \beta_4D_{4i} + \beta_5D_{5i} +\epsilon_i
$$

Wobei

| Beruf | D3 | D4 |
| --- | --- | --- |
| Physiker | 1 | 0 |
| Zahnarzt | 0 | 1 |
| Biologe | 0 | 0 |

| Geschlecht | D5 |
| --- | --- |
| Mann | 0 |
| Frau | 1 |

eine mögliche Kodierung für die Variablen wäre.

In [9]:
accidents = pd.read_csv("Unfaelle.txt", sep=" ")

print(f"Datensatz:\n{accidents.head()}")

Datensatz:
   Lnr  Unfall Geschlecht     Beruf  Alter  Fahrpraxis
0    1       0       Mann  Physiker     31          12
1    2       0       Frau  Physiker     37          18
2    3       0       Frau  Physiker     38          18
3    4       0       Frau  Physiker     44          25
4    5       0       Frau  Physiker     52          33


In [10]:
model = sm.formula.glm(formula="Unfall ~ Alter + Fahrpraxis + C(Beruf) + C(Geschlecht)", data=accidents, family=sm.families.Binomial()).fit()

print(f"Kodierung der Koeffizienten: {model.model.exog_names}")

Kodierung der Koeffizienten: ['Intercept', 'C(Beruf)[T.Physiker]', 'C(Beruf)[T.Zahnarzt]', 'C(Geschlecht)[T.Mann]', 'Alter', 'Fahrpraxis']


**B:** Berechnen Sie in R die Koeffizientenschätzer.

In [11]:
print(model.params)

Intercept               -1.828690
C(Beruf)[T.Physiker]    -0.643373
C(Beruf)[T.Zahnarzt]     1.488117
C(Geschlecht)[T.Mann]   -1.169886
Alter                    0.263490
Fahrpraxis              -0.638461
dtype: float64


**C:** Das Modell M habe die Parameter $β_0, . . . , β_p$. Welche anschauliche Bedeutung haben die exponierten Parameter $exp(β_0), . . . , exp(β_p)$?

Werden die Parameter exponiert, so erhält man die Odds-Ratios oder relative Risiken.

**Für logistische Regressionen**: Exponentierte Koeffizienten zeigen das Odds-Ratio an, wenn eine Prädiktorvariable um 1 Einheit erhöht wird, während alle anderen Variablen konstant bleiben.

**Für andere Modelle**: Exponentierte Koeffizienten können als relative Risiken oder Hazard-Ratios interpretiert werden.

**Allgemeine Interpretation**:
- Ein exponentierter Koeffizient > 1 zeigt eine positive Assoziation zwischen Prädiktor und Outcome an.
- Ein exponentierter Koeffizient < 1 zeigt eine negative Assoziation an.
- Ein exponentierter Koeffizient = 1 zeigt keine Assoziation an.

**Aufgabe 12**

Berechnen Sie aus dem Modell M (Aufgabe 11) das Odds und die Wahrscheinlichkeit
- eines 30-jährigen Zahnarztes mit 10 Jahren Fahrpraxis,
- einer 29-jährigen Zahnärztin mit 11 Jahren Fahrpraxis,
- einer 50-jährigen Biologin mit 30 Jahren Fahrpraxis,

einen Unfall zu verursachen.

In [12]:
A_data = pd.DataFrame({'Alter': [30], 'Fahrpraxis': [10], 'Beruf': ['Zahnarzt'], 'Geschlecht': ['Mann']})
B_data = pd.DataFrame({'Alter': [29], 'Fahrpraxis': [11], 'Beruf': ['Zahnarzt'], 'Geschlecht': ['Frau']})
C_data = pd.DataFrame({'Alter': [50], 'Fahrpraxis': [30], 'Beruf': ['Biologe'], 'Geschlecht': ['Frau']})

A_pred = model.predict(A_data)
B_pred = model.predict(B_data)
C_pred = model.predict(C_data)

print("Probs:")
print(A_pred[0])
print(B_pred[0])
print(C_pred[0])

def odds(prob):
    return prob/(1-prob)

print("Odds:")
print(odds(A_pred[0]))
print(odds(B_pred[0]))
print(odds(C_pred[0]))

Probs:
0.5024090585060654
0.5689504251412866
0.0004062982606426319
Odds:
1.0096828873083286
1.3199187711245821
0.00040646340601751174


**Aufgabe 13**
Betrachten Sie das logistische Regressionsmodell
$$
L : \text{Unfall} ∼ \text{Alter}
$$

für die Daten in Unfaelle.txt.

**A:** Berechnen Sie mit der Funktion `glm`, `coef` und `vcov` die Parameterschätzer
und die Varianz-Kovarianz Matrix V des Modells L.

In [15]:
model_log = sm.formula.glm(formula="Unfall ~ Alter", data=accidents, family=sm.families.Binomial()).fit()

print(model_log.params)

cov_matrix = model_log.cov_params()
print(cov_matrix)

Intercept    8.794495
Alter       -0.327340
dtype: float64
           Intercept     Alter
Intercept   0.485038 -0.014985
Alter      -0.014985  0.000473


**B:** Geben Sie eine Formel an, die log(Odds) in Abhängigkeit von Alter
berechnet.

$$
\log \Bigg ( \frac{p}{1-p} \Bigg ) = \beta_0 + \beta_1 \text{Alter}
$$

**C:** Berechnen Sie mit der Varianz-Kovarianz Matrix die Standardabweichung von log(Odds).

$$
Var(\log (Odds)) = Var(\beta_0 + \beta_1 \cdot Alter) = Cov(\beta_0 + \beta_1 \cdot Alter, \beta_0 + \beta_1 \cdot Alter) = Var(\beta_0) + 2 \cdot Alter \cdot Cov(\beta_1, \beta_0) + Alter^2 \cdot Var(\beta_1)
$$

**Aufgabe 14**

Berechnen Sie mit der Funktion glm das Modell
$$K : \text{Unfall} ∼ \text{Geschlecht} \cdot \text{Beruf}$$
für die Daten in Unfaelle.txt. Die rechte Seite $\text{Geschlecht} \cdot \text{Beruf}$ in der
Modelldefinition bedeutet, dass neben Beruf und Geschlecht auch noch alle
Produkte von Geschlecht mit den zu Beruf gehörenden Dummy-Variablen
in die Modellgleichung aufgenommen werden. Diese Produkte definieren die
Interaktion zwischen Beruf und Geschlecht.
Wie viele Modellparameter $β_0, . . . , β_p$ hat das Modell K? Welche anschauliche
Bedeutung haben die exponierten Modellparameter $exp(\hat{β}_0), . . . , exp(\hat{β}_p)$

Zunächst erinnern wir uns:

$$
\frac{1}{1-p} = exp(\beta_0 + \beta_1 \text{Mann} + ...)
$$

$$
\log \frac{1}{1-p} = \beta_0 + \beta_1 \text{Mann} + ...
$$

Wir können es manuell durch Ableitung auflösen, aber auch für jede mögliche Multiplikation eine Dummy-Variable erstellen.

In [16]:
model_k = sm.formula.glm("Unfall ~ C(Geschlecht) * C(Beruf)", data=accidents, family=sm.families.Binomial()).fit()

model_k.params

Intercept                                    -3.158120
C(Geschlecht)[T.Mann]                        -0.519988
C(Beruf)[T.Physiker]                         -0.333026
C(Beruf)[T.Zahnarzt]                          0.894464
C(Geschlecht)[T.Mann]:C(Beruf)[T.Physiker]   -0.419683
C(Geschlecht)[T.Mann]:C(Beruf)[T.Zahnarzt]   -0.435233
dtype: float64

Es werden 6 Parameter benötigt. Neben den Koeffizienten für die einzelnen Parametern müssen auch noch sämtlich mögliche einzelne Interaktionen betrachtet werden, also $2 \cdot 3 = 6$

Die exponierten Parameter 

In [17]:
print(model_k.params.apply(lambda x: np.exp(x)))

Intercept                                     0.042506
C(Geschlecht)[T.Mann]                         0.594528
C(Beruf)[T.Physiker]                          0.716752
C(Beruf)[T.Zahnarzt]                          2.446025
C(Geschlecht)[T.Mann]:C(Beruf)[T.Physiker]    0.657255
C(Geschlecht)[T.Mann]:C(Beruf)[T.Zahnarzt]    0.647114
dtype: float64


geben den Faktor an, um den sich die Wahrscheinlichkeit eines Unfalls verschiebt, falls die jeweilige Variable sich um eine Einheit verschiebt, während alle anderen Faktoren konstant gehalten werden. 

**Aufgabe 15**

In den Jahren 2000, 2001 und 2002 werden an einem Autobahnabschnitt $k_{2000} = 2$, $k_{2001} = 5$ und $k_{2002} = 0$ Unfälle beobachtet. Es wird
angenommen, dass die Anzahlen der Unfälle $X_1$, $X_2$ und $X_3$ in den Jahren
2000, 2001 und 2002 unabhängig und poissonverteilt mit Erwartungswert $\lambda$
sind.

**A:** Geben Sie die Likelihood-Funktion $L(λ)$ und die Loglikelihood-Funktion $l(λ)$ an.

$$
L(\lambda) = L(\lambda | k_1, ..., k_n) = \prod_{i=1}^{n} e^{-\lambda} \frac{\lambda^{k_i}}{k_i!}
$$

$$
= \Bigg[e^{-\lambda} \frac{\lambda^{2}}{2!}\Bigg] \cdot \Bigg[e^{-\lambda} \frac{\lambda^{5}}{5!}\Bigg] \cdot \Bigg[e^{-\lambda} \frac{\lambda^{0}}{0!}\Bigg] = \Bigg[e^{-\lambda} \frac{\lambda^{2}}{2}\Bigg] \cdot \Bigg[e^{-\lambda} \frac{\lambda^{5}}{120}\Bigg] \cdot \Bigg[e^{-\lambda}\Bigg]
$$

Loglikelihood-Funktion:

$$
l(\lambda) = \sum_{i=1}^{n} (-\lambda + k_i \log (\lambda) - \log (k_i !))
$$

$$
l(\lambda) = (-\lambda + 2 \log (\lambda) - \log (2 !)) + (-\lambda + 5 \log (\lambda) - \log (5 !)) + (-\lambda + 0 \log (\lambda) - \log (0 !))
$$

**B:** Berechnen Sie $l'(λ)$ und daraus den Maximum-Likelihoodschätzer $\hat{λ}$.

$$
\hat{\lambda} = \frac{1}{n} \sum_{i=1}^{n} k_i
$$

$$
\hat{\lambda} = \frac{1}{3} \Big ( 2 + 5 + 0 \Big ) = \frac{7}{3}
$$

**C:** Berechnen Sie $l''(λ)$ und daraus den Varianzschätzer $Var(\hat{\lambda})$ von $\hat{λ}$.

$$
l''(\lambda) = \frac{\delta^2 l}{\delta \lambda ^2} = -\frac{n}{\lambda ^2} = -\frac{7}{\lambda ^2}
$$

**D:** Geben Sie für $\hat{λ}$ das 95% Konfidenzintervall an. Benutzen Sie die Formel $\hat{λ} ± 2 \cdot \sqrt{Var(\hat{λ})}$

$$
Var(\hat{\lambda}) = \hat{\text{Cov}} (\hat{\lambda}) = ... = \frac{1}{l''(\lambda)} = \frac{1}{\frac{7}{\lambda ^2}} = \frac{\lambda ^2}{7}
$$